In [1]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2Model
import pandas as pd
from tqdm import tqdm
import string



In [2]:
# Load GPT2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2', output_hidden_states=True)

In [3]:
punctuation_map = dict((ord(char), None) for char in string.punctuation)  #引入标点符号，为下步去除标点做准备

In [4]:
df_paragraphs = pd.read_csv('/Users/carina/Downloads/courses/final thesis/dataset/annotated text.csv')

df_keywords_tfidf5 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_tfidf_5.csv')
df_keywords_tfidf5 = df_keywords_tfidf5.drop(df_keywords_tfidf5.columns[0],axis=1)

df_keywords_tfidf10 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_tfidf_10.csv')
df_keywords_tfidf10 = df_keywords_tfidf10.drop(df_keywords_tfidf10.columns[0],axis=1)

df_keywords_tfidf15 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_tfidf_15.csv')
df_keywords_tfidf15 = df_keywords_tfidf15.drop(df_keywords_tfidf15.columns[0],axis=1)

df_cleanedtext = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/annotated_cleaned.csv')
df_cleanedtext = df_cleanedtext.drop(df_cleanedtext.columns[0],axis=1)

df_origintext = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/annotated_originalwords.csv')
df_origintext = df_origintext.drop(df_origintext.columns[0],axis=1)


In [5]:
df_emvecOfkw_tfidf5 = pd.DataFrame()
df_emvecOfkw_tfidf10 = pd.DataFrame()
df_emvecOfkw_tfidf15 = pd.DataFrame()

In [6]:
def get_all_index(lst=None, item=''):
    return [i for i in range(len(lst)) if lst[i] == item]

def get_index(word,tokenlist):
    
    indices = []
    
    for item in tokenlist:
        if item in word:
            index0 = tokenlist.index(item)
            index1 = index0 + 1

            if index1 != len(tokenlist):
                tem_2combine_token = item + tokenlist[index1]
                
                if tem_2combine_token in word:
                    index2 = index1 + 1

                    if index2 != len(tokenlist):
                        tem_3combine_token = tem_2combine_token + tokenlist[index2]
                        
                        if tem_3combine_token in word:
                            indices.append(index0)
                            indices.append(index1)
                            indices.append(index2)
                        else:
                            if tem_2combine_token == word:
                                indices.append(index0)
                                indices.append(index1)

                    else:
                        if tem_2combine_token == word:
                            indices.append(index0)
                            indices.append(index1)
    return indices

In [7]:
heads = df_paragraphs.columns.values.tolist()
for head in heads:
    allnumOfori = 0
    allnumOfkw = 0
    j = 0
    paras = []
    paras = df_paragraphs[head].tolist()
    
    keyword_tfidf10 = []
    keyword_tfidf10 = df_keywords_tfidf10[head].tolist()
    
    clean_word = []
    clean_word = df_cleanedtext[head].tolist()
    
    original_word = []
    original_word = df_origintext[head].tolist()
    
    words_embeddings = []
    for i in tqdm(range(len(paras))):
        
        paragraphs = paras[i]
        words = keyword_tfidf10[i]
        cleanword = clean_word[i].strip('[').strip(']')
        cleanword = cleanword.translate(punctuation_map)
        cleanword = cleanword.split(' ')
        
        originalword = original_word[i].strip('[').strip(']')
        originalword = originalword.translate(punctuation_map)
        originalword = originalword.split(' ')

        paragraphs = paragraphs.lower()
        paragraphs = paragraphs.translate(punctuation_map)
        
        # Tokenize the paragraph and convert the tokens to IDs
        input_ids = torch.tensor(tokenizer.encode(paragraphs, add_special_tokens=True)).unsqueeze(0)
        tokens = [token.replace('Ġ', '') for token in tokenizer.convert_ids_to_tokens(input_ids[0])]

        # Generate embeddings for the input IDs using the BERT model
        outputs = model(input_ids)
        
        # Get the hidden states of the last layer from the model
        if len(outputs) >= 3:
            last_layer_hidden_states = outputs[2][-1]
        else:
            last_layer_hidden_states = outputs.last_hidden_state
        
        words = words.strip('[').strip(']')
        words = words.translate(punctuation_map)
        words = words.split(' ')
        
        word_embeddings = []
        
        for word in words:
            useless_token = 0
            # Find the index of the first occurrence of the word in the tokens
            if word in tokens:
                index = tokens.index(word)
                # Extract the corresponding embedding vector from the GPT model output
                embedding = last_layer_hidden_states[0][index].detach().numpy()
                word_embeddings.append(embedding)
            else :
                j += 1
                index = get_all_index(cleanword,word)
                origins = []
                for ind in index:
                    origins.append(originalword[ind])   
                    
                originem = []
                for ori in origins:
                    if ori in tokens:
                        index = tokens.index(ori)
                        # Extract the corresponding embedding vector from the GPT model output
                        embedding = last_layer_hidden_states[0][index].detach().numpy()
                        originem.append(embedding)
                    else:
                        index_list = get_index(ori,tokens)
                        if len(index_list) != 0:
                            tem_emOfori = []
                            for index in index_list:
                                embedding = last_layer_hidden_states[0][index].detach().numpy()
                                tem_emOfori.append(embedding)
                            tem_emOfori = np.mean(tem_emOfori, axis=0)
                            originem.append(tem_emOfori)
                        else : 
                            allnumOfori += 1

                if len(originem) != 0 :
                    
                    originem = np.mean(originem, axis=0)
                    word_embeddings.append(originem)
                else:
                    allnumOfkw += 1
                    originem = [0.0]*768
                    word_embeddings.append(originem)

        words_embeddings.append(word_embeddings)
        
    df_emvecOfkw_tfidf10[head] = words_embeddings
    print(j)
    print('allnumOfori : ')
    print(allnumOfori)
    print('allnumOfkw :')
    print(allnumOfkw)


100%|███████████████████████████████████████████| 480/480 [04:43<00:00,  1.69it/s]


3361
allnumOfori : 
403
allnumOfkw :
212


100%|███████████████████████████████████████████| 480/480 [04:54<00:00,  1.63it/s]


3458
allnumOfori : 
396
allnumOfkw :
201


100%|███████████████████████████████████████████| 480/480 [04:15<00:00,  1.88it/s]


...

3329
allnumOfori : 
322
allnumOfkw :
145


100%|███████████████████████████████████████████| 480/480 [04:24<00:00,  1.82it/s]


3350
allnumOfori : 
394
allnumOfkw :
169


100%|███████████████████████████████████████████| 480/480 [04:24<00:00,  1.81it/s]

3379
allnumOfori : 
418
allnumOfkw :
213


In [8]:
df_emvecOfkw_tfidf10.to_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/emvec_GPT_keywords_tfidf10.csv')

In [13]:
# Define the paragraph and words to generate embeddings for
df_paragraphs = pd.read_csv('/Users/carina/Downloads/courses/final thesis/dataset/annotated text.csv')

df_keywords_YAKE5 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_YAKE_5.csv')
df_keywords_YAKE5 = df_keywords_YAKE5.drop(df_keywords_YAKE5.columns[0],axis=1)

df_keywords_YAKE10 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_YAKE_10.csv')
df_keywords_YAKE10 = df_keywords_YAKE10.drop(df_keywords_YAKE10.columns[0],axis=1)

df_keywords_YAKE15 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_YAKE_15.csv')
df_keywords_YAKE15 = df_keywords_YAKE15.drop(df_keywords_YAKE15.columns[0],axis=1)

#df_cleanedtext = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/annotated_cleaned.csv')
#df_cleanedtext = df_cleanedtext.drop(df_cleanedtext.columns[0],axis=1)


In [14]:
df_emvecOfkw_YAKE5 = pd.DataFrame()
df_emvecOfkw_YAKE10 = pd.DataFrame()
df_emvecOfkw_YAKE15 = pd.DataFrame()
#df_emvecOfallwords = pd.DataFrame()

In [15]:
heads = df_paragraphs.columns.values.tolist()

for head in heads:
    if heads.index(head) != 6:
        allnum = 0
        j = 0
        uselesskws = 0
        paras = []
        paras = df_paragraphs[head].tolist()

        keyword_YAKE10 = []
        keyword_YAKE10 = df_keywords_YAKE10[head].tolist()

        words_embeddings = []
        for paragraphs,words in tqdm(zip(paras,keyword_YAKE10)):

            #paragraphs = paragraphs.lower()
            paragraphs = paragraphs.translate(punctuation_map)

            # Tokenize the paragraph and convert the tokens to IDs
            input_ids = torch.tensor(tokenizer.encode(paragraphs, add_special_tokens=True)).unsqueeze(0)
            tokens = [token.replace('Ġ', '') for token in tokenizer.convert_ids_to_tokens(input_ids[0])]

            # Generate embeddings for the input IDs using the BERT model
            outputs = model(input_ids)

            # Get the hidden states of the last layer from the model
            if len(outputs) >= 3:
                last_layer_hidden_states = outputs[2][-1]
            else:
                last_layer_hidden_states = outputs.last_hidden_state

            words = words.strip('[').strip(']')
            words = words.translate(punctuation_map)
            words = words.split(' ')

            word_embeddings = []

            for word in words:
                useless_token = 0
                word = word.translate(punctuation_map)
                # Find the index of the first occurrence of the word in the tokens
                if word in tokens:
                    index = tokens.index(word)
                    # Extract the corresponding embedding vector from the GPT model output
                    embedding = last_layer_hidden_states[0][index].detach().numpy()
                    word_embeddings.append(embedding)
                else :
                    j = j + 1
                    inputid = torch.tensor(tokenizer.encode(word, add_special_tokens=True)).unsqueeze(0)
                    tem_token = [token.replace('Ġ', '') for token in tokenizer.convert_ids_to_tokens(inputid[0])]
                    tem_em = []
                    combinetoken = str()
                    for item in tem_token:
                        if item in tokens:
                            index = tokens.index(item)
                            # Extract the corresponding embedding vector from the GPT model output
                            embedding = outputs.last_hidden_state[0][index].detach().numpy()
                            tem_em.append(embedding)
                        else:
                            useless_token += 1
                            combinetoken += item
                            if combinetoken in tokens:
                                index = tokens.index(combinetoken)
                                # Extract the corresponding embedding vector from the GPT model output
                                embedding = outputs.last_hidden_state[0][index].detach().numpy()
                                tem_em.append(embedding)
                                combinetoken = str()
                                useless_token = 0
                    if len(combinetoken) != 0:
                        allnum = allnum + useless_token
                    if len(tem_em)!=0:       
                        tem_em = np.mean(tem_em, axis=0)
                        word_embeddings.append(tem_em)
                    else:
                        uselesskws += 1
                        tem_em = [0.0]*768
                        word_embeddings.append(tem_em)
                        print('terrible1')

            words_embeddings.append(word_embeddings)
        df_emvecOfkw_YAKE10[head] = words_embeddings
        print(j)
        print(allnum)
        print(uselesskws)


37it [00:19,  2.10it/s]

terrible1


38it [00:19,  2.18it/s]

terrible1


39it [00:20,  2.21it/s]

...

terrible1


462it [04:19,  2.89it/s]

terrible1


480it [04:25,  1.81it/s]

642
131
48


In [11]:
df_emvecOfkw_YAKE10.to_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/emvec_GPT2_keywords_YAKE10.csv')